In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 30
pd.options.display.max_columns = 100

In [ ]:
rawdata = pd.read_csv('rawsim2.csv',sep=',',decimal='.',encoding='ANSI',keep_default_na = False, na_values = [''])

slicemeta = pd.read_csv('slicemeta2.csv',sep=',',decimal='.',encoding='ANSI')

metadata = pd.read_csv('aggregations_metadata4.csv',sep=',',decimal='.',encoding='ANSI')
agglist = pd.read_csv('aggregations_agglist5.csv',sep=',',decimal='.',encoding='ANSI')
varcomb = pd.read_csv('varcomb2.csv',sep=',',decimal='.',encoding='ANSI')

metadataR = pd.read_csv('aggregations_metadataR.csv',sep=',',decimal='.',encoding='ANSI')
agglistR = pd.read_csv('aggregations_agglistR.csv',sep=',',decimal='.',encoding='ANSI')
seglistR = pd.read_csv('aggregations_segmlist.csv',sep=',',decimal='.',encoding='ANSI')

timesincemeta = pd.read_csv('timesincemeta2.csv',sep=',',decimal='.',encoding='ANSI')

issomemeta = pd.read_csv('issomemeta.csv',sep=',',decimal='.',encoding='ANSI')

intermeta = pd.read_csv('intermeta.csv',sep=',',decimal='.',encoding='ANSI')

catmeta = pd.read_csv('catmeta.csv',sep=',',decimal='.',encoding='ANSI')
catmetab = pd.read_csv('catmetab.csv',sep=',',decimal='.',encoding='ANSI')

In [ ]:
import sys
sys.path.insert(0, '../..')
import scoring
import scoring.feature_engineering

In [ ]:
### RELOADING MODULE FOR TESTING PURPOSES
#import importlib
#importlib.reload(scoring)
#importlib.reload(scoring.feature_engineering_files.feature_engineering_from_slice)
#importlib.reload(scoring.feature_engineering_files.categorical_features)
#importlib.reload(scoring.feature_engineering_files.interactions)
#importlib.reload(scoring.feature_engineering_files.is_something)
#importlib.reload(scoring.feature_engineering_files.slicer_order_assigner)
#importlib.reload(scoring.feature_engineering_files.time_since_calc)
#importlib.reload(scoring.feature_engineering_files.utils)
#importlib.reload(scoring.feature_engineering)

In [ ]:
display(rawdata)

In [ ]:
from scoring.feature_engineering import OrderAssigner

oa = OrderAssigner(time_name = 'TIME', 
                   time_max_name = 'TIME_APPLICATION',
                   time_granularity = 'months',
                   history_length = 6,
                   time_format = '%Y-%m-%d %H:%M:%S',
                   partition_name = None)
oa.fit(rawdata)
orderdata = oa.transform(rawdata)

In [ ]:
print(oa.strsql_)
display(orderdata)

In [ ]:
from scoring.feature_engineering import FeatureEngineeringFromSlice

fe1 = FeatureEngineeringFromSlice(id_name = 'ID_APPLICATION',
                                  time_name = 'TIME_ORDER',
                                  metadata = metadataR,
                                  agglist = agglistR, 
                                  varcomb = None,
                                  segm = seglistR,
                                  max_time = 6,
                                  min_fill_share = 0)
fe1.fit(orderdata)
transrawdata = fe1.transform(orderdata)

In [ ]:
print(fe1.strsql_)
display(transrawdata)

In [ ]:
from scoring.feature_engineering import Slicer

sl = Slicer(id_name = 'ID_APPLICATION',
            time_name = 'TIME',
            time_max_name = 'TIME_APPLICATION',
            slicemeta = slicemeta,
            time_granularity = 'months',
            history_length = 6,
            time_format = '%Y-%m-%d %H:%M:%S')
sl.fit(rawdata)
newdata = sl.transform(rawdata)

In [ ]:
print(sl.strsql_)
display(newdata)

In [ ]:
from scoring.feature_engineering import FeatureEngineeringFromSlice

fe2 = FeatureEngineeringFromSlice(id_name = 'ID_APPLICATION',
                                  time_name = 'TIME_ORDER',
                                  metadata = metadata,
                                  agglist = agglist,
                                  varcomb = varcomb,
                                  segm = None,
                                  max_time = 6,
                                  min_fill_share = 0)
fe2.fit(newdata)
transdata = fe2.transform(newdata)

In [ ]:
print(fe2.strsql_)
display(transdata)

In [ ]:
from scoring.feature_engineering import TimeSinceCalc

tsc = TimeSinceCalc(id_name = 'ID_APPLICATION',
                    time_name = 'TIME',
                    time_max_name = 'TIME_APPLICATION',
                    timesincemeta = timesincemeta,
                    time_format='%Y-%m-%d %H:%M:%S',
                    keyword='entry')
tsc.fit(rawdata)
ds = tsc.transform(rawdata)

In [ ]:
print(tsc.strsql_)
display(ds)

In [ ]:
from scoring.feature_engineering import IsSomething

iss = IsSomething(id_name = 'ID_APPLICATION',
                  time_name = 'TIME_ORDER',
                  issomemeta = issomemeta,
                  max_time = np.inf)
iss.fit(newdata)
issomedata = iss.transform(newdata)

In [ ]:
print(iss.strsql_)
display(issomedata)

In [ ]:
from scoring.feature_engineering import CategoricalFeatures

ct = CategoricalFeatures(id_name = 'ID_APPLICATION', 
                         from_type = 'raw', 
                         catmeta = catmeta,
                         slice_name = 'TIME_ORDER',
                         time_name = 'TIME',
                         time_max_name = 'TIME_APPLICATION',
                         time_format = '%Y-%m-%d %H:%M:%S')
ct.fit(orderdata)
catdata = ct.transform(orderdata)

In [ ]:
print(ct.strsql_)
display(catdata)

In [ ]:
from scoring.feature_engineering import CategoricalFeatures

ctb = CategoricalFeatures(id_name = 'ID_APPLICATION', 
                          from_type = 'slice', 
                          catmeta = catmetab, 
                          slice_name = 'TIME_ORDER',
                          time_name = None,
                          time_max_name = None,
                          time_format = '%Y-%m-%d %H:%M:%S')
ctb.fit(newdata)
catdatab = ctb.transform(newdata)

In [ ]:
print(ctb.strsql_)
display(catdatab)

In [ ]:
from scoring.feature_engineering import Interactions

itr = Interactions(id_name = 'ID_APPLICATION',
                   intermeta = intermeta)
itr.fit(rawdata)
itrdata = itr.transform(rawdata)

In [ ]:
print(itr.strsql_)
display(itrdata)